# A Model of *Angry Birds* in VPython
This is a notebook to model an *Angry Birds*-style game in VPython.

It aims to replicate the same mechanics as Angry Birds, modelling projectile motion, collisions and torque. 
The user inputs an initial velocity and angle above the ground for a particle to try to topple over a target some distance away in the x-direction. This model only considers motion in 2-D.

The system follows a classical model, making several assumptions about the conservation of kinetic energy and momentum, amongst other things. I based the game mechanics on the equations outlined in Section 2 of the Final Assignment Script (Dash, 2019). 

The game mechanics follow an algorithm, which is outlined in Figure 2 of the Final Assignment Script (Dash, 2019).

To implement the algorithm, I drew out a plan before beginning coding, planning which loops I would use. I decided to use a base *while* loop , to which I assigned the variable *win == False*, which ensured the code would repeat each time the user failed to topple the target. Only when the user topples the target does *win = True*, which leads the code to exit the loop, and lets the user know that they have won.

Within the base loop, I implemented a series of nested *while* and *if* loops. These are explained below.

In [1]:
# import necessary modules
import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, arrow

<IPython.core.display.Javascript object>

## Strategy
The base *while* loop matched the step 1 in the flow chart in Figure 2 of the Script. I added a nested *while* loop to match step 2 in the chart, the bird's motion through the air.

Within this *while* loop I added an *if* loop to match step 3: "Does it hit the target?".
    If the bird doesn't hit the target, the code exits back to the initial while loop and repeats the process again, requesting user input.
    If it does hit the target, it calculates the applied and restoring torques to determine whether or not the target will topple. If it doesn't topple, it exits the loop, back to the base *while* loop. If it does topple, *win == True*, ending the game.

In [2]:
# set the scene
scene = canvas(width=640, height=480, center=vector(8,5,0),range=8)
ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)
g = 9.81        # define gravitational acceleration, m/s^2

# create target using box object [1], use random() to randomise x-position between 5=<x>=15
target = box(pos=vector(5+10*random(),1,0), width=0.5, height=2, length=0.5, color=color.cyan)
target.mass = 100 #set mass = 100kg

# create platform using box object
platform = box(pos=vector(0.25,random(),0), width=0.5, height=0.1, length=0.5, color=color.cyan)
#platform = box(pos=vector(0.25,0.5,0), width=0.5, height=0.1, length=0.5, color=color.cyan)

# create bird, set to launch from platform position
bird = sphere(radius=0.3, color=color.red)
bird.pos = (platform.pos+vector(0,0.3,0))
bird.mass = 0.1

# initialise no. of attempts at zero
num = 0

# define labels
# use pixel position so that labels remained at fixed position
# visible=False to start, become visible at appropriate time
L_1 = label(pixel_pos=True, pos=vector(100,350,0), box=False, visible=False, text="Almost! Try again")
L_2 = label(pixel_pos=True, pos=vector(100,400,0), box=False, visible=False, text="Total attempts:" + str(num))
L_3 = label(pixel_pos=True, pos=vector(500,400,0), box=False, visible=False, text="YOU WIN!")

# create arrow for momentum
mom = arrow(pos=bird.pos,shaftwidth=0.1, visible=True)

# define parameters for projection
step = 0.0001   # time interval for loop animation, s
x0 = 0.0        # initial bird x-coordinate, m
y0 = vector(platform.pos+vector(0,0.3,0)).y

# initialise base while loop
win = False

# enter base while loop
while win == False:
    scene.waitfor("redraw") # to control visibility
    num = num + 1
    # input initial conditions (angle and speed)
    dtheta = float(input("Input the initial angle in degrees: "))
    theta = np.radians(dtheta)
    v0 = float(input("Input the initial speed in metres/second: "))
    t = 0 # initialise time
    y = y0 # initialise y
    x = x0 # initialise x
    L_1.visible = False
    L_2.text = "Total attempts: " + str(num)
    L_2.visible = True
    collision = False  # introduce collision variable
    while y > 0:      # has the ball hit the ground yet?
        t = t + step    # increase time by timestep
        rate(5000)  # increase rate to obtain smoother parabola
        x = x0 + v0*t*np.cos(theta)     # x position
        y = y0 + v0*t*np.sin(theta) - 0.5*(g*t**2)     # y position 
        bird.pos = vector(x,y,0)    # new position of bird
        p_x = bird.mass*v0*np.cos(theta)  # x-momentum, from eqn (4)
        p_y = bird.mass*v0*np.sin(theta)-bird.mass*g*t   # y-momentum, from eqn (5)
        mom.pos = bird.pos     # match arrow motion to bird motion
        mom.axis = vector(p_x,p_y,0)   # arrow to match momentum vector
        # enter if loop. round position components to 2dp 
        if (     round(bird.pos.x,2) == (round(target.pos.x-0.25,2))
             and (round(bird.pos.y,2) <= round(target.height,2))
            ):
            collision = True
            dt = 0.01             # define parameters for collision
            d_a = vector(bird.pos.x,bird.pos.y,0) - vector(target.pos.x+target.width,0,0)
            F_app = vector(p_x/dt,p_y/dt,0)   # eqn (11)
            T_app = cross(F_app,d_a)          # eqn (9)
            T_res = target.mass*g*(target.width/2) # restoring torque, eqn (8)
            y=0
            if T_app.mag > T_res:
                # print values rounded to 2dp
                print("\nHeight of impact: ", round(bird.pos.y,2),"m")
                print("\nMomentum of bird at impact: ", round(mom.axis.mag,2), "kg•m/s")
                print("\nApplied torque: ", round(T_app.mag,2), "Nm")
                print("\nRestoring torque: ", round(T_res,2), "Nm")
                win = True
    if collision == False:
        L_1.visible = True
        mom.visible = False
        print("The bird landed",np.abs(target.pos.x - bird.pos.x), "m away from the target")
if win == True:
    L_3.visible = True   # 
    mom.visible = False  # arrow disappears as bird transfers all its momentum to target

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Input the initial angle in degrees: 30
Input the initial speed in metres/second: 8
The bird landed 6.723255807711188 m away from the target
Input the initial angle in degrees: 20
Input the initial speed in metres/second: 10
The bird landed 5.377897980554724 m away from the target
Input the initial angle in degrees: 5
Input the initial speed in metres/second: 100

Height of impact:  1.61 m

Momentum of bird at impact:  9.99 kg•m/s

Applied torque:  1661.43 Nm

Restoring torque:  245.25 Nm


## Challenges in the Project
I faced several challenges in this project, for example:

1) *Changing the visibility of the label() objects.* From [3] I found the scene.waitfor() function, which I used to vary the visibility of the labels at different stages of the game.

2) Keeping the labels at fixed positions throughout the game. From [4] I found that by setting *pixel_pos=True* I could keep the labels at fixed positions, which were dependent on the pixel coordinates, rather than on the reference frame of the equations of motion.

Ultimately the VPython documentation proved very helpful.

## Possible Improvements to the Physics
The game is very basic. I have made many assumptions in the physics to make it easier to code. There are several improvements that could be made so that the game would more accurately represent a typical classical system.

1) The bird is assumed to act as a point-like particle. To create a more accurate model, the bird could be modelled to have angular momentum about its centre of mass. To implement this, a vector quantity could be introduced that acts rotationally, centred about the bird's centre of mass. Furthermore, in the code, the collision is between the centres of mass of the bird and the target, not between their outer edges. This could be improved by changing the position vectors of the collision to match the outer edges.

2) It is assumed that the target cannot slide along the ground. Instead, a friction force F=µR could be introduced, where µ is the coefficient of friction between the target and the ground, and R is the reaction force of the ground on the target. µ could change once the target starts sliding, as is observed in classical systems.

3) The bird doesn't hit the ground after the collision. This could be implemented in the code by introducing another variable *ground = False*, which becomes *ground = True* after the collision, so that it falls.

4) The animation ends after the ball either collides with the target or hits the ground. To create a more accurate model, the animation could continue, and the bird could bounce, gradually losing kinetic energy until it comes to rest. This could be implemented by modelling the collision between the bird and the ground as inelastic and using a coefficient of restitution to find the kinetic energy lost after each bounce. I would use a while loop to iterate the loss of energy.

## References
[1] Dash, L. (2019) *PHAS0007: Final Assignment Script*. UCL Moodle. Accessed from https://moodle.ucl.ac.uk/mod/resource/view.php?id=1587821.

[2] GlowScript Project, "Simple 3D Programming Using VPython" - https://www.glowscript.org/docs/VPythonDocs/VisualIntro.html

[3] GlowScript Project,  VPython canvas() Documentation - 
https://www.glowscript.org/docs/GlowScriptDocs/canvas.html

[4] GlowScript Project, VPython label() Documentation - https://www.glowscript.org/docs/VPythonDocs/label.html